# Combining Python, SQL DBs and AWS

## Aim: Connect to your DB on AWS through python, and execute queries.  


# Using Python with MYSQL DB

In [5]:
# make sure we have the package installed
!pip install mysql-connector-python

In [6]:
## importing 'mysql.connector' 
import mysql.connector 

**mysql.connector is similar to workbench**

In [7]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = "localhost", 
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [8]:
import config

In [9]:
config.user

'sbourdeau930'

In [10]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

print(cnx) # it will print a connection object if everything is fine


In [11]:
cursor = cnx.cursor()

## SQL Commands: Data Definition Language Commands (DDL)
This section will discuss the commands through which you can define your database. The commands are as follows:

- CREATE
- DROP
- TRUNCATE
- ALTER
- BACKUP DATABASE

### CREATE

This statement is used to create a table or a database.



#### The ‘CREATE DATABASE’ Statement:
As the name suggests, this statement is used to create a database.

**Syntax**

`CREATE DATABASE DatabaseName;`

**Example**

`CREATE DATABASE Employee;`


#### The **‘CREATE TABLE’** Statement

This statement is used to create a table.

**Syntax**

`CREATE TABLE TableName (
Column1 datatype,
Column2 datatype,
Column3 datatype,
....
ColumnN datatype
);`

**Example**

`CREATE TABLE Employee_Info
(
EmployeeID int,
EmployeeName varchar(255),
Emergency ContactName varchar(255),
PhoneNumber int,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`

You can also create a table using another table. Refer the below sytax and example:

#### The ‘CREATE TABLE AS’ Statement
**Syntax**

`CREATE TABLE NewTableName AS
SELECT Column1, column2,..., ColumnN
FROM ExistingTableName
WHERE ....;`

**Example**

`CREATE TABLE ExampleTable AS
SELECT EmployeeName, PhoneNumber
FROM Employee_Info;`

[MYSQL Data Types](http://www.mysqltutorial.org/mysql-data-types.aspx)

### DROP
This statement is used to drop an existing table or a database.

#### The ‘DROP DATABASE’ Statement
This statement is used to drop an existing database. When you use this statement, complete information present in the database will be lost.

**Syntax**

`DROP DATABASE DatabaseName;`

**Example**

`DROP DATABASE Employee;`

#### The ‘DROP TABLE’ Statement
This statement is used to drop an existing table. When you use this statement, complete information present in the table will be lost.

**Syntax**

`DROP TABLE TableName;`

**Example**

`DROP Table Employee_Info;`

### TRUNCATE
This command is used to delete the information present in the table but does not delete the table. So, once you use this command, your information will be lost, but not the table.

**Syntax**

`TRUNCATE TABLE TableName;`

**Example**

`TRUNCATE Table Employee_Info;`

### The ‘ALTER TABLE’ Statement
This statement is used to add, delete, modify columns in an existing table.

#### The ‘ALTER TABLE’ Statement with ADD/DROP COLUMN
You can use the ALTER TABLE statement with ADD/DROP Column command according to your need. If you wish to add a column, then you will use the ADD command, and if you wish to delete a column, then you will use the DROP COLUMN command.

**Syntax**
`ALTER TABLE TableName
ADD ColumnName Datatype;`

`ALTER TABLE TableName
DROP COLUMN ColumnName;`

**Example**

`ALTER TABLE Employee_Info
ADD BloodGroup varchar(255);`
 
`ALTER TABLE Employee_Info
DROP COLUMN BloodGroup ;`

#### The ‘ALTER TABLE’ Statement with ALTER/MODIFY COLUMN
This statement is used to change the datatype of an existing column in a table.

**Syntax**

`ALTER TABLE TableName
ALTER COLUMN ColumnName Datatype;`

**Example**
 
`ALTER TABLE Employee_Info
ADD DOB year;
ALTER TABLE Employee_Info
ALTER DOB date;`

### BACKUP DATABASE
This statement is used to create a full backup of an existing database.

**Syntax**
`BACKUP DATABASE DatabaseName
TO DISK = 'filepath';`

**Example**

`BACKUP DATABASE Employee
TO DISK = 'C:UsersSahitiDesktop';`


## SQL Commands: Constraints Used In Database
Constraints are used in a database to specify the rules for data in a table. The following are the different types of constraints:

- NOT NULL
- UNIQUE
- CHECK
- DEFAULT
- INDEX


### NOT NULL
This constraint ensures that a column cannot have a NULL value.

**Example**

 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
NOT NULL on ALTER TABLE
 
`ALTER TABLE Employee_Info
MODIFY PhoneNumber int NOT NULL;`

### UNIQUE
This constraint ensures that all the values in a column are unique.

**Example**

UNIQUE on Create Table
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL UNIQUE,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
UNIQUE on Multiple Columns
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255),
CONSTRAINT UC_Employee_Info UNIQUE(Employee_ID, PhoneNumber)
);` --- does not add a new column, just says grouping of columns must be unique
 
UNIQUE on ALTER TABLE
 
`ALTER TABLE Employee_Info
ADD UNIQUE (Employee_ID);`
 
To drop a UNIQUE constraint
 
`ALTER TABLE  Employee_Info
DROP CONSTRAINT UC_Employee_Info;`

## Creating DB

In [12]:
from mysql.connector import errorcode

In [13]:
db_name = 'employees'

In [14]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
    #above is python code that is going to connect to SQL queries w/ cursor.execute
    #cursor knows to take SQL query and send it to server to run code
    #try & except is error handling within python
    #format takes in database and tries to run it
    #if failed, except will tell you error and why it failed
    #above is a function to create a database

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating tables

In [15]:
DB_NAME = 'employees'

TABLES = {} #use dictionary to save diff create statements for each table
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT," #index automatically created
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [16]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [17]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close() 
cnx.close()
#cursor and connection are closed after the tables are run

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


## Inserting Data

In [18]:
#since we closed the connection, we need to reinstatiate/reopen it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [ ]:
# cursor = create_cnx()

In [19]:
from datetime import date, datetime, timedelta

tomorrow = datetime.now().date() + timedelta(days=1) 
#datetime.now() takes the current date and time down to nanosecond, date() eliminates everything minus the date

add_employee = ("INSERT INTO employees " 
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))

#order of data_employee is dynamically replaced into query 

In [20]:
data_employee

('Geert',
 'Vanderkelen',
 datetime.date(2019, 10, 24),
 'M',
 datetime.date(1977, 6, 14))

In [21]:
cursor

In [22]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()
#when doing an insert, database is held off from being saved until data is committed



In [23]:
emp_no

5

## Insert Many

cursor.executemany(operation, seq_of_params)

In [24]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

#connection wasn't closed so we don't need to reopen here, just for reference

In [25]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data) 
#takes statement and data as tuples and inserts all as individual rows in database

In [26]:
cnx.commit()

In [27]:
cursor.close()
cnx.close()

## Querying the DB

In [28]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [29]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = date(2005, 1, 1)
hire_end = date(2010, 12, 31)

cursor.execute(query, (hire_start, hire_end)) #adds the dates into %s


In [30]:
for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

#loop over cursor to get results
#no last name in query, so last_name shows up blank

, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010
, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010


## Applied: Working with our student data

### Import our student data

In [31]:
import json

In [32]:


f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [33]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

### Create a table for our student info

In [34]:
data[21]

{'name': 'Stephanie Bourdeau',
 'dob': '1993-09-30',
 'birthplace': 'Brooklyn, NY',
 'siblings': 2,
 'years_in_nyc': 26.1,
 'favorite_food': 'NYC pizza'}

In [35]:
create_query = """
CREATE TABLE students (
    student_id INT(7) NOT NULL UNIQUE AUTO_INCREMENT,
    name varchar(20) NOT NULL, 
    dob date NOT NULL,
    birthplace varchar(50) NOT NULL,
    siblings int(32) NOT NULL,
    years_in_nyc float(32) NOT NULL,
    favorite_food varchar(50) NOT NULL
    )Engine=InnoDB
"""

#format is column_name, type, not null(or null),
#the number after floar or varchar just helps allocate space (32 or 7 characters, etc)

In [36]:
cursor.execute(create_query)

ProgrammingError: 1050 (42S01): Table 'students' already exists

### Insert the student data into the table

In [37]:
data[0]

{'name': 'Sean Abu Wilson',
 'dob': '1985-02-06',
 'birthplace': 'Birmingham, AL',
 'siblings': 2,
 'years_in_nyc': 8.4,
 'favoritefood': 'oatmeal raisin cookies'}

In [38]:
students_list = []

for student in data:
    if 'favoritefood' in student.keys():
        student["favorite_food"] = student["favoritefood"]
        
    student_tuple = (student['name'], student['dob'],
                    student['siblings'], student['birthplace'],
                    student['years_in_nyc'], student['favorite_food'])
    students_list.append(student_tuple)

In [40]:
#insert data into table
insert_stmt = "INSERT INTO students (name, dob, siblings, birthplace, years_in_nyc, favorite_food) VALUES (%s, %s, %s, %s, %s, %s)"

In [41]:
cursor.executemany(insert_stmt, students_list)
cnx.commit()

In [ ]:
#
# keys_list = []
# for student in data:
#     print(student['name'])
#     keys = student.keys()
#     for word in keys:
#         if word not in keys_list:
#             keys_list.append(word)
# print (keys_list)

In [ ]:
student = data[0]

### Write queries to answer the following questions:

**Questions**
- Which student is the youngest?
- Which student's birthday is closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- What are the most popular foods among the class?

[Utilizing the date data type in MySQL](http://www.mysqltutorial.org/mysql-date/)

In [44]:
cursor.execute("""SELECT * FROM students LIMIT 5;""")